<h1>Q1. Running Elastic</h1>

es_client.info(): 與 Python 應用程式集成，以根據 Elasticsearch 叢集狀態自動執行監控、日誌記錄或警報等任務。



In [ ]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

try:
    es_info = es_client.info()
    print(es_info)
except Exception as e:
    print(f"Error: {e}")

es_client.info()

第一步：curl localhost:9200 是一種從命令列檢查 Elasticsearch 的簡單而直接的方法 <br>
es_client.info():與 Python 應用程式集成，以根據 Elasticsearch 叢集狀態自動執行監控、日誌記錄或警報等任務。<br>
es_client = Elasticsearch('http://localhost:9200')，初始化 es_client 一次以連接到 Elasticsearch<br>


What's the version.build_hash value?<br>

terminal-- curl localhost:9200
"build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",

Q1: 42f05b9372a9a4a470db3b52817899b99a76ee73

<h1>Q2. Indexing the data
</h1>

In [ ]:
pip install requests

In [ ]:
pip install elasticsearch


In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# es_client.indices.create(index=index_name, body=index_settings)
es_client.indices.exists(index=index_name)


Q2. Indexing the data <br>
Which function do you use for adding your data to elastic?<br>

index

<h1>Q3. Searching</h1>


In [ ]:
query = 'How do I execute a command in a running docker container?'

In [ ]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

search_results = es_client.search(index=index_name, body=search_query)


In [ ]:
search_results['hits']['hits'][0]['_score']


Q3: What's the score for the top ranking result? <br>
A: 84.05


<h1>Q4. Filtering</h1>


In [ ]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

search_results = es_client.search(index=index_name, body=search_query)

In [ ]:
search_results['hits']['hits'][2]


Q4: Return 3 results. What's the 3rd question returned by the search engine? <br>
A: How do I copy files from a different folder into docker container’s working directory?

<h1>Q5. Building a prompt
</h1>

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [ ]:
context_pieces = []

for hit in search_results['hits']['hits']:
    doc = hit['_source']
    # 使用 context_template 格式化文件。
    # 假設 context_template 是一個字串模板，它使用 doc 中的欄位來建立格式化輸出。
    # **doc 語法將字典 doc 解包為關鍵字參數，然後使用關鍵字參數取代 context_template 中的佔位符
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)

In [ ]:
prompt = prompt_template.format(question=query, context=context)



In [ ]:
len(prompt)

Q4: What's the length of the resulting prompt? (use the len function) <br>
A: 1462

In [ ]:
print(prompt)

<h1>Q6. Tokens</h1>

In [ ]:
pip install tiktoken


In [ ]:
import openai

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")


In [ ]:
len(encoding.encode(prompt))


In [ ]:
tokens = encoding.encode(prompt)
tokens

In [ ]:
encoding.decode_single_token_bytes(tokens[0])


<h1>Bonus: generating the answer (ungraded)</h1>

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": query}]
)

response.choices[0].message.content

In [87]:
answer = response.choices[0].message.content

<h1>Bonus: calculating the costs (ungraded)</h1>

In [ ]:
send = 150
receive = 250
input_cost = send * 0.005/1000
output_cost = receive * 0.015/1000
costs = (input_cost+output_cost)*1000

print(costs)

In [89]:
send_tokens = len(encoding.encode(prompt))
receive_token = len(encoding.encode(answer))

input_cost = send_tokens * 0.005/1000
output_cost = receive_token * 0.015/1000
costs = input_cost+output_cost

print(costs)


0.00917
